# Graded Assignment 3: 9 to 5

Time to show off your SQL skills! For each question, copy the SQL query you used and make note of the answer.

## The Dataset

For this assignment, you will be using the Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results which can be found on [Kaggle](https://www.kaggle.com/datasets/bls/employment).

## Business Issue

You are working for the Bureau of Labor Statistics with the United States government and have been approached by your boss with an important meeting request. You have been asked by your supervisor to meet with Dolly Parton whose nonprofit is looking to shed light on the state of employment in the United States. As part of the 9 to 5 project, their research is focused on production and nonsupervisory employees and how those employees fare compared to all employees in the United States. While the data the BLS collects from the CES is publicly available, Dolly Parton and her colleagues need your assistance navigating the thousands of rows in each table in LaborStatisticsDB.

## About the Dataset

This dataset comes directly from the Bureau of Labor Statistics’ Current Employment Survey (CES). Here are some things you need to know:

1. The industry table contains an NAICS code. This is different from the industry code. NAICS stands for North American Industry Classification System.
1. Series ID is composed of multiple different codes. CES stands for Current Employment Survey, the name of the survey which collected the data. The industry code as specified by the BLS and the data type code as specified in the datatype table.

## Set Up

To connect to the database, use the same connection info used during the SQL lessons. 

For the assignment, we will be using `LaborStatisticsDB`.

## Database Exploration

To start with, let’s get to know the database further.

1. Use this space to make note of each table in the database, the columns within each table, each column’s data type, and how the tables are connected. You can write this down or draw a diagram. Whatever method helps you get an understanding of what is going on with `LaborStatisticsDB`.
   
   To add a photo, diagram or document to your file, drop the file into the folder that holds this notebook.  Use the link button to the right of the  </> symbol in the gray part of this cell, the link is just the name of your file.

In [ ]:
annual_2016
    id	smallint
    series_id	nvarchar
    year	smallint
    period	nvarchar
    value	float
    footnote_codes	nvarchar
    original_file	nvarchar
datatype
    data_type_code	tinyint
    data_type_text	nvarchar
footnote
    footnote_code	nvarchar
    footnote_text	nvarchar
industry
    id	smallint
    industry_code	bigint
    naics_code	nvarchar
    publishing_status	nvarchar
    industry_name	nvarchar
    display_level	float
    selectable	nvarchar
    sort_sequence	float
january_2017
    id	bigint
    series_id	nvarchar
    year	smallint
    period	nvarchar
    value	float
    footnote_codes	nvarchar
    original_file	nvarchar
PERIOD
    period_code	nvarchar
    month_abbr	nvarchar
    month	nvarchar
seasonal
    industry_code	nvarchar
    seasonal_text	nvarchar
series 
    series_id	nvarchar
    supersector_code	bigint
    industry_code	nvarchar
    data_type_code	bigint
    seasonal	nvarchar
    series_title	nvarchar
supersector
    supersector_code	tinyint
    supersector_name	nvarchar

--I've also made the ERD diagram for this. 


2. What is the datatype for women employees?

In [ ]:
SELECT *
FROM LaborStatisticsDB.dbo.datatype
WHERE data_type_text like '%Women%'
 
-- Ans 10

3. What is the series id for  women employees in the commercial banking industry in the financial activities supersector?

In [ ]:
SELECT *
FROM LaborStatisticsDB.dbo.series
WHERE industry_code =55522110 AND supersector_code = 55 AND series_title = 'Women employees'

-- Ans: CES5552211010 and CEU5552211010

## Aggregate Your Friends and Code some SQL

Put together the following:

1. How many employees were reported in 2016 in all industries? Round to the nearest whole number.

In [ ]:

SELECT distinct COUNT(id)
FROM LaborStatisticsDB.dbo.annual_2016

 --Ans: 29042

2. How many women employees were reported in 2016 in all industries? Round to the nearest whole number. 

In [ ]:

SELECT COUNT(series_id)
FROM LaborStatisticsDB.dbo.annual_2016
where series_id like '%10'    --- 10 is the data type for women employees

--Ans 1372


3. How many production/nonsupervisory employees were reported in 2016? Round to the nearest whole number. 

In [ ]:

SELECT COUNT(*) as Total_Prod_Sup_Emp
FROM LaborStatisticsDB.dbo.annual_2016
WHERE SUBSTRING(series_id,13,1) = 6

--ANS:3590

4. In January 2017, what is the average weekly hours worked by production and nonsupervisory employees across all industries?

In [ ]:
SELECT 
  LEFT(AVG(value),5) as avg_weekly_hours
    FROM LaborStatisticsDB.dbo.january_2017
    WHERE SUBSTRING(series_id, 12,2)= 07

-- ANS: 36.05

5. What is the total weekly payroll for production and nonsupervisory employees across all industries in January 2017? Round to the nearest penny.

In [ ]:

SELECT SUM(value) as total_weekly_pay
    FROM LaborStatisticsDB.dbo.january_2017
    WHERE SUBSTRING(series_id, 12,2)= 81

--ANS: 84559120

6. In January 2017, for which industry was the average weekly hours worked by production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
USE LaborStatisticsDB

SELECT TOP 1
    i.industry_name,
   AVG(j.value)  AS avg_weekly_hours
FROM january_2017 AS j
JOIN series AS s 
    ON j.series_id = s.series_id
JOIN industry AS i 
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 07
GROUP BY i.industry_name
ORDER BY AVG(j.value) DESC;



SELECT TOP 1
    i.industry_name,
   AVG(j.value)  AS avg_weekly_hours
FROM january_2017 AS j
JOIN series AS s 
    ON j.series_id = s.series_id
JOIN industry AS i 
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 07
GROUP BY i.industry_name
ORDER BY AVG(j.value) ASC;

--Ans: Motor vehicle power train components	49.8 -- Highest
--     Fitness and recreational sports centers	16.7 --- Lowest


7. In January 2021, for which industry was the total weekly payroll for production and nonsupervisory employees the highest? Which industry was the lowest?

In [ ]:
--- taking Jan 2017, no Jan 2021 data in the Database
USE LaborStatisticsDB

SELECT TOP 1
    i.industry_name,
    j.value AS total_weekly_payroll
FROM january_2017 AS j
JOIN series AS s 
    ON j.series_id = s.series_id
JOIN industry AS i 
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 82
ORDER BY j.value DESC;


USE LaborStatisticsDB

SELECT TOP 1
    i.industry_name,
    j.value AS total_weekly_payroll
FROM january_2017 AS j
JOIN series AS s 
    ON j.series_id = s.series_id
JOIN industry AS i 
    ON s.industry_code = i.industry_code
WHERE s.data_type_code = 82
ORDER BY j.value ASC;

--HIghest Industry: Total private	74498171
-- Lowest Industry: Coin-operated laundries and drycleaners	10079

## Join in on the Fun

Time to start joining! You can choose the type of join you use, just make sure to make a  note!

1. Join `annual_2016` with `series` on `series_id`. We only want the data in the `annual_2016` table to be included in the result.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *

USE LaborStatisticsDB
GO

SELECT  TOP 50 a.*
FROM annual_2016 AS  a
INNER JOIN series AS s ON a.series_id = s.series_id;


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

USE LaborStatisticsDB
GO

SELECT  TOP 50 a.*
FROM annual_2016 AS  a
INNER JOIN series AS s ON a.series_id = s.series_id
ORDER BY id;

Commands completed successfully.

Total execution time: 00:00:00.019

2. Join `series` and `datatype` on `data_type_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY series id


USE LaborStatisticsDB
GO

SELECT  TOP 50 *
FROM series AS s 
INNER JOIN datatype AS dt ON s.data_type_code = dt.data_type_code
ORDER BY series_id;

3. Join `series` and `industry` on `industry_code`.

In [ ]:
-- Limiting rows returned from query, uncomment the line below to start on your query!
-- SELECT TOP 50 *


-- Uncomment the line below when you are ready to run the query, leaving it as your last!
-- ORDER BY id

USE LaborStatisticsDB
GO

SELECT  TOP 50 *
FROM series AS s 
INNER JOIN industry AS ind ON s.industry_code = ind.industry_code
ORDER BY s.series_id, ind.id;


## Subqueries, Unions, Derived Tables, Oh My!

1. Write a query that returns the `series_id`, `industry_code`, `industry_name`, and `value` from the `january_2017` table but only if that value is greater than the average value for `annual_2016` of `data_type_code` 82.

In [ ]:
SELECT distinct(jan.series_id), s.industry_code, jan.value, ind.industry_name
FROM LaborStatisticsDB.dbo.january_2017 AS jan
INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
INNER JOIN LaborStatisticsDB.dbo.industry AS ind ON s.industry_code = ind.industry_code
where jan.value > (
    SELECT AVG(an.[value])
      FROM LaborStatisticsDB.dbo.annual_2016 AS an   
      INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = an.series_id
      INNER JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code =s.data_type_code
      WHERE dt.data_type_code = 82
);

**Optional Bonus Question:** Write the above query as a common table expression!

In [ ]:
WITH cte_avg_value_annual AS (
  SELECT AVG(an.[value]) AS an_value
      FROM LaborStatisticsDB.dbo.annual_2016 AS an   
      INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = an.series_id
      INNER JOIN LaborStatisticsDB.dbo.datatype AS dt ON dt.data_type_code =s.data_type_code
      WHERE dt.data_type_code = 82
)

SELECT distinct(jan.series_id), s.industry_code, jan.value, ind.industry_name
FROM LaborStatisticsDB.dbo.january_2017 AS jan
INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
INNER JOIN LaborStatisticsDB.dbo.industry AS ind ON s.industry_code = ind.industry_code
where jan.value > ( SELECT * FROM cte_avg_value_annual);

2. Create a `Union` table comparing average weekly earnings of production and nonsupervisory employees between `annual_2016` and `january_2017` using the data type 30.  Round to the nearest penny.  You should have a column for the average earnings and a column for the year, and the period.

In [ ]:
SELECT ROUND(AVG(jan.value), 1) AS avg_weekly_earnings, jan.year,jan.period 
FROM LaborStatisticsDB.dbo.january_2017 AS jan 
INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = jan.series_id
WHERE s.data_type_code =30
GROUP BY jan.year, jan.[period]

UNION 

SELECT ROUND(AVG(ann.value), 1) AS avg_weekly_earnings, ann.year,ann.period 
FROM LaborStatisticsDB.dbo.annual_2016 AS ann 
INNER JOIN LaborStatisticsDB.dbo.series AS s ON s.series_id = ann.series_id
WHERE s.data_type_code =30
GROUP BY ann.[year], ann.[period]

## Summarize Your Results

With what you know now about the  Bureau of Labor Statistics (BLS) Current Employment Survey (CES) results and working with the Labor Statistics Database, answer the following questions. Note that while this is subjective, you should include relevant data to back up your opinion.

1. During which time period did production and nonsupervisory employees fare better?

During January 2017

2. In which industries did production and nonsupervisory employees fare better?

Reinsurance carrier for the Jan 2017.

Pipeline transportation for annual 2016

I used these parameter for my results: 30 / 31 — AVERAGE WEEKLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES.

3. Now that you have explored the datasets, is there any data or information that you wish you had in this analysis?

Yes, Instead of Jan 2017, if we have annual 2017, the better analysis. Also, data has different datatype for same group without any explanation.